# Using the cache

First make sure that the dependencies are installed:

```bash
pip install -r requirements-dev.txt
```

Then run redis using docker compose:

```bash
docker compose -f docker-compose.redis.yml up -d
```

Then run the notebook. 

If you want to view the data in redis, you can use the redis-cli:

```bash
brew install redis
redis-cli -p 6380
```

In [1]:
import fsspec
import xarray as xr
from redis import Redis
from redis_fsspec_cache.reference import RedisCachingReferenceFileSystem

redis = Redis(host="localhost", port=6380)

new_cached_fs = RedisCachingReferenceFileSystem(
    redis=redis,
    expiry_time=60,
    fo='s3://nextgen-dmac-cloud-ingest/nos/ngofs2/nos.ngofs2.fields.best.nc.zarr', 
    remote_protocol='s3', 
    remote_options={'anon':True}, 
    target_protocol='s3', 
    target_options={'anon':True}, 
    asynchronous=True, 
)

m = new_cached_fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", backend_kwargs=dict(consolidated=False), 
                      chunks={}, drop_variables=['Itime2'])
ds

ImportError: cannot import name 'RedisAsyncCachingFilesystem' from 'redis_fsspec_cache.reference' (/Users/matthewiannucci/Developer/redis-fsspec-cache/redis_fsspec_cache/reference.py)

In [2]:
%%time
ds.lat.values

CAT: ['lat/0']
CACHE HIT: ['lat/0']
CPU times: user 195 ms, sys: 38.2 ms, total: 233 ms
Wall time: 380 ms


array([21.789427, 21.7978  , 21.81    , ..., 30.50772 , 30.50776 ,
       30.507713], dtype=float32)

In [2]:
%%time
len(ds.nv.values)

CAT: ['nv/0.0']
CACHE HIT: ['nv/0.0']
CPU times: user 223 ms, sys: 57.1 ms, total: 280 ms
Wall time: 538 ms


3

In [2]:
%%time

ds.zeta.isel(time=slice(0,10)).values

CAT: ['zeta/9.0']CAT: ['zeta/8.0']

CAT: ['zeta/7.0']
CAT: ['zeta/6.0']
CAT: ['zeta/5.0']
CAT: ['zeta/4.0']
CAT: ['zeta/3.0']
CAT: ['zeta/2.0']
CAT: ['zeta/1.0']
CAT: ['zeta/0.0']
CACHE HIT: ['zeta/8.0']
CACHE HIT: ['zeta/9.0']
CACHE HIT: ['zeta/5.0']
CACHE HIT: ['zeta/3.0']
CACHE HIT: ['zeta/0.0']
CACHE HIT: ['zeta/1.0']
CACHE HIT: ['zeta/2.0']
CACHE HIT: ['zeta/4.0']
CACHE HIT: ['zeta/7.0']
CACHE HIT: ['zeta/6.0']
CPU times: user 236 ms, sys: 67.8 ms, total: 304 ms
Wall time: 490 ms


array([[0.12666899, 0.12503888, 0.12107819, ..., 0.7163431 , 0.7143663 ,
        0.716661  ],
       [0.10200123, 0.09967896, 0.09507909, ..., 0.7435788 , 0.74137074,
        0.74403507],
       [0.1794594 , 0.17622934, 0.17010154, ..., 0.71228325, 0.710277  ,
        0.712741  ],
       ...,
       [0.27834344, 0.27714086, 0.27271524, ..., 0.791554  , 0.7890122 ,
        0.791849  ],
       [0.24430923, 0.24382065, 0.2409046 , ..., 0.75616574, 0.7539281 ,
        0.7563978 ],
       [0.1446265 , 0.14446583, 0.14271444, ..., 0.82348686, 0.82094306,
        0.82362497]], dtype=float32)

In [3]:
len(fs.cat(['nv/0.0']))

1